# Import Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import classification_report, log_loss   
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold

# Data Preprocessing

In [2]:
df = pd.read_csv("datanewfinal02.csv") 

In [3]:
df_dom = df.drop(["diff%","Ez","Medium","Hard"], axis=1)

In [4]:
df_dom

,Algebra,Adv Math,PsDA,Geometry,dom%
0,0.71,0.52,0.67,0.67,4
1,0.52,0.86,0.78,0.89,19
2,0.71,0.90,0.56,0.89,16
3,0.67,0.81,0.78,0.44,15
4,0.62,0.95,0.44,0.44,14
...,...,...,...,...,...
1995,0.71,0.48,0.89,0.44,17
1996,0.81,0.76,0.44,0.78,1
1997,0.67,0.90,0.78,0.67,0
1998,0.90,1.00,1.00,0.44,21


In [5]:
df_diff = df.drop(["Algebra","Adv Math","PsDA","Geometry","dom%"], axis=1)

In [6]:
df_diff

,Ez,Medium,Hard,diff%
0,0.88,0.91,0.58,10
1,0.63,0.78,0.33,3
2,0.56,0.91,0.17,9
3,0.75,0.56,0.50,2
4,1.00,0.81,0.33,4
...,...,...,...,...
1995,0.81,0.69,0.42,2
1996,0.56,0.66,0.42,3
1997,0.88,0.84,0.08,4
1998,1.00,0.44,0.50,2


In [7]:
def scale_dataset(dataframe, oversample = False):
    X = dataframe[dataframe.columns[:-1]].values   #set x as every columns except the last
    y = dataframe[dataframe.columns[-1]].values    #set y to be the last column

    scaler = StandardScaler()   #fit and transform x so it scales with other inputs
    X = scaler.fit_transform(X)

    if oversample:
        ros = RandomOverSampler()
        X,y = ros.fit_resample(X,y)        #Take the less class & keep sampling so that the smaller class would match
    
    data = np.hstack((X, np.reshape(y, (-1, 1))))        #horizontally stack arrays & reshape y into 2 dimensional (-1 being the length of y)

    return data, X, y, scaler

# Data Splitting

+ 1: Domain Classification

In [8]:
train1, test1 = np.split(df_dom.sample(frac=1),[int(0.8*len(df_dom))])

In [9]:
train1 = pd.DataFrame(train1)  # Convert numpy array to pandas DataFrame
train1, X1_train, y1_train,scaler_dom = scale_dataset(train1, oversample=True)
test1, X1_test, y1_test,scaler01 = scale_dataset(test1, oversample=False)

In [10]:
X1_train

array([[ 1.32317198, -1.53601969,  0.30999771, -0.3098758 ],
       [ 1.63271304,  0.76247154, -0.84543636, -0.3098758 ],
       [-0.71979905,  1.30685104, -0.26771933, -0.3098758 ],
       ...,
       [ 0.14691593, -0.68920713,  1.46543178,  0.85808952],
       [ 0.14691593, -1.29407324,  1.46543178,  1.44207218],
       [-0.41025799, -0.38677408,  1.46543178,  0.85808952]])

+ 2: Difficulty Classification

In [11]:
train2, test2 = np.split(df_diff.sample(frac=1),[int(0.8*len(df_diff))])

In [12]:
train2 = pd.DataFrame(train2)  # Convert numpy array to pandas DataFrame

In [13]:
train2, X2_train, y2_train, scaler_diff = scale_dataset(train2, oversample=True)
test2, X2_test, y2_test, scaler02 = scale_dataset(test2, oversample=False)

In [14]:
X2_train

array([[ 1.19211757,  0.98256706,  1.3208981 ],
       [-1.50574398, -0.14218493, -1.37907611],
       [-0.70864852, -0.31977735, -0.02908901],
       ...,
       [ 1.19211757,  1.69293674, -0.28622941],
       [ 0.82422736,  1.69293674, -0.28622941],
       [ 1.19211757,  0.98256706, -0.28622941]])

# Hyperparameter Tuning

+ 1: Domain Classification

In [15]:
# Logistic Regression model
log_reg1 = LogisticRegression(random_state=1111)

# Parameter grid
param_grid = {'C': [0.01, 0.1, 1, 10, 100, 1000]}

# Stratified K-Fold cross-validation
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Grid Search with fixed random state
grid_search = GridSearchCV(log_reg1, param_grid, cv=cv, scoring='accuracy', n_jobs=-1)
grid_search.fit(X1_train, y1_train)

# Best parameters
print("Best Parameters:", grid_search.best_params_)

Best Parameters: {'C': 1000}


+ 2: Difficulty Classification

In [16]:
# Logistic Regression model
log_reg2 = LogisticRegression(random_state=1111)

# Parameter grid
param_grid = {'C': [0.01, 0.1, 1, 10, 100,1000]}

# Stratified K-Fold cross-validation
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Grid Search with fixed random state
grid_search = GridSearchCV(log_reg1, param_grid, cv=cv, scoring='accuracy', n_jobs=-1)
grid_search.fit(X2_train, y2_train)

# Best parameters
print("Best Parameters:", grid_search.best_params_)

Best Parameters: {'C': 1000}


# Model Deploy

+ 1: Domain Classification

In [17]:
lg_dom_model = LogisticRegression(C=1000, random_state=1111)
lg_dom_model = lg_dom_model.fit(X1_train, y1_train)

y1_pred = lg_dom_model.predict(X1_test)

+ 2: Difficulty Classification

In [18]:
lg_diff_model = LogisticRegression(C=1000, random_state=1111)
lg_diff_model = lg_diff_model.fit(X2_train, y2_train)

y2_pred = lg_diff_model.predict(X2_test)

# Model Evaluation

+ 1: Domain Classification

In [19]:
print(classification_report(y1_train, lg_dom_model.predict(X1_train)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        97
           1       1.00      1.00      1.00        97
           2       1.00      1.00      1.00        97
           3       1.00      1.00      1.00        97
           4       1.00      1.00      1.00        97
           5       1.00      1.00      1.00        97
           6       1.00      1.00      1.00        97
           7       1.00      1.00      1.00        97
           8       1.00      1.00      1.00        97
           9       1.00      1.00      1.00        97
          10       1.00      1.00      1.00        97
          11       1.00      1.00      1.00        97
          12       1.00      1.00      1.00        97
          13       1.00      1.00      1.00        97
          14       1.00      1.00      1.00        97
          15       1.00      1.00      1.00        97
          16       1.00      1.00      1.00        97
          17       1.00    

In [20]:
print(classification_report(y1_test,y1_pred)) 

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        18
           1       0.96      1.00      0.98        22
           2       1.00      1.00      1.00        21
           3       1.00      1.00      1.00        15
           4       0.94      1.00      0.97        16
           5       1.00      1.00      1.00        11
           6       1.00      1.00      1.00        18
           7       1.00      1.00      1.00        10
           8       1.00      1.00      1.00        18
           9       0.92      1.00      0.96        12
          10       0.90      0.82      0.86        11
          11       1.00      1.00      1.00        11
          12       1.00      0.94      0.97        16
          13       1.00      1.00      1.00        14
          14       1.00      0.93      0.97        15
          15       1.00      1.00      1.00        21
          16       1.00      0.94      0.97        16
          17       1.00    

In [21]:
# Assuming X_train and y_train are your training features and labels
# Assuming X_test is your test features
# Assuming model is your logistic regression model object

# Fit the logistic regression model
lg_dom_model.fit(X1_train, y1_train)

# Predict probabilities for the test data
y1_pred_prob = lg_dom_model.predict_proba(X1_test)

# The predicted probabilities will be a 2D array where each row corresponds to a sample in X_test
# and each column contains the probability of that sample belonging to each class
# In binary classification, the second column typically represents the probability of the positive class

# Now you can calculate the log-loss
logloss = log_loss(y1_test, y1_pred_prob)
print("Logarithmic Loss:", logloss)

Logarithmic Loss: 0.029266259061158248


+ 2: Difficulty Classification

In [22]:
print(classification_report(y2_train, lg_diff_model.predict(X2_train)))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       154
           1       1.00      1.00      1.00       154
           2       1.00      1.00      1.00       154
           3       1.00      1.00      1.00       154
           4       1.00      1.00      1.00       154
           5       1.00      1.00      1.00       154
           6       1.00      1.00      1.00       154
           7       1.00      1.00      1.00       154
           8       1.00      0.99      1.00       154
           9       1.00      1.00      1.00       154
          10       1.00      1.00      1.00       154
          11       1.00      1.00      1.00       154

    accuracy                           1.00      1848
   macro avg       1.00      1.00      1.00      1848
weighted avg       1.00      1.00      1.00      1848



In [23]:
print(classification_report(y2_test,y2_pred)) 

              precision    recall  f1-score   support

           0       0.88      1.00      0.93        28
           1       1.00      1.00      1.00        27
           2       1.00      0.95      0.97        38
           3       0.86      0.97      0.91        38
           4       1.00      0.92      0.96        37
           5       1.00      0.95      0.97        38
           6       0.94      0.97      0.95        32
           7       1.00      0.96      0.98        28
           8       0.97      0.89      0.93        37
           9       0.90      1.00      0.95        26
          10       1.00      0.89      0.94        36
          11       0.95      1.00      0.97        35

    accuracy                           0.95       400
   macro avg       0.96      0.96      0.96       400
weighted avg       0.96      0.95      0.96       400



In [24]:
# Assuming X_train and y_train are your training features and labels
# Assuming X_test is your test features
# Assuming model is your logistic regression model object

# Fit the logistic regression model
lg_diff_model.fit(X2_train, y2_train)

# Predict probabilities for the test data
y2_pred_prob = lg_diff_model.predict_proba(X2_test)

# The predicted probabilities will be a 2D array where each row corresponds to a sample in X_test
# and each column contains the probability of that sample belonging to each class
# In binary classification, the second column typically represents the probability of the positive class

# Now you can calculate the log-loss
logloss = log_loss(y2_test, y2_pred_prob)
print("Logarithmic Loss:", logloss)

Logarithmic Loss: 0.09781714774372585


# Outputs

In [25]:
def feedback_diff(predicted):
    if predicted == 0:
        return "Bạn đang sai nhiều ở các câu có độ khó được đánh giá ỏ mức Easy, trong khi đó bạn lại trả lời đúng ở các câu có độ khó được đánh giá ở mức Medium và Hard. Nguyên nhân của việc này thường là do sự thiếu cẩn thận, hoặc thiếu sót trong kiến thức cơ bản. Để khắc phục, bạn hãy ôn lại kĩ lý thuyết cũng như cẩn thận hơn trong quá trình làm bài. "
    elif predicted == 1:
        return "Bạn đã nắm khá chắc các kiến thức cơ bản, và đã có thể vận dụng để làm các câu có độ khó được đánh giá ở mức Medium và Hard. Tuy nhiên các câu hỏi có độ khó Hard có vẻ vẫn đang hơi quá sức với bạn tại thời điểm này. Bạn nên tập trung luyện tập các câu hỏi có độ khó Medium để thành thục chúng. Và để cải thiện, hãy xem lại các phần kiến thức bạn còn sai sót và luyện tập các bài tập liên quan đến các dạng bài đó."
    elif predicted == 2:
        return "Bạn đã nắm khá chắc các kiến thức cơ bản, tuy nhiên chưa thể vận dụng tốt để làm các câu có độ khó được đánh giá ở mức Medium và Hard. Để cải thiện, hãy xem lại các phần kiến thức bạn còn sai sót và luyện tập các bài tập liên quan đến các dạng bài đó nhé! Trước hết hãy quay lại luyện tập các câu hỏi có độ khó Medium, tránh các lỗi có thể tránh do không cẩn thận hoặc thiếu sót trong kiến thức cơ bản."
    elif predicted == 3:
        return "Bạn đang làm khá tốt ở các câu hỏi có độ khó Medium, tuy nhiên lại sai ở các câu hỏi có độ khó được đánh giá ở mức Easy và Hard. Nguyên nhân có thể là do sự thiếu cẩn thận trong quá trình làm bài. Để cải thiện, hãy thử làm bài một cách chậm rãi để cải thiện độ chính xác cho các câu hỏi Easy và Medium, sau đó hãy luyện tập các câu hỏi có độ khó Hard để nâng cao khả năng giải quyết vấn đề của mình."
    elif predicted == 4:
        return "Bạn đã nắm khá chắc các kiến thức cơ bản và có thể xử lý các câu hỏi có độ khó được đánh giá ở mức Easy và Medium rất tốt, tuy nhiên mức độ thể hiện ở các câu có độ khó Hard chỉ ở mức trung bình. Để đạt được điểm số cao hơn, hãy luyện tập thêm các câu hỏi có mức độ khó được đánh giá Hard để cải thiện tư duy và kĩ năng giải quyết các bài toán khó."
    elif predicted == 5:
        return "Tuy đã nắm được các kiến thức cơ bản và có thể xử lý các câu hỏi có độ khó Hard, bạn lại đang gặp vấn đề với các câu hỏi có độ khó Medium. Nguyên nhân có thể là do sự thiếu cẩn thận hoặc thiếu sót kiến thức của các domain. Để cải thiện, hãy xem lại các phần kiến thức bạn còn sai sót và luyện tập các bài tập liên quan đến các dạng bài đó. "
    elif predicted == 6:
        return "Trình độ của bạn hiện đang ở mức Beginner, nghĩa là bạn chưa có quá nhiều tiếp xúc với toán SAT. Hãy học kĩ các kiến thức nền cơ bản của khóa học, nắm chắc lý thuyết và luyện tập nhiều từ các câu hỏi dễ đến khó để cải thiện và đạt được điểm số mong muốn."
    elif predicted == 7:
        return "Tuy làm tốt các câu hỏi có độ khó Hard, bạn lại đang gặp nhiều sai sót với các câu hỏi có độ khó Easy và Medium. Nguyên nhân chính có lẽ là do sự thiếu cẩn thận. Để khắc phục, bạn hãy luyện 1 số đề thi hoặc làm bài tính thời gian để cải thiện kĩ năng quản lý thời gian của mình, đồng thời học lại kĩ phần lý thuyết cũng như bài tập của khóa học bạn."
    elif predicted == 8:
        return "Về mặt tổng quát, bạn đã làm rất tốt ở các câu hỏi thuộc cả 3 độ khó Easy, Medium và Hard. Để có thể đạt được kết quả tốt thường xuyên hơn, bạn có thể luyện tập với các bài kiểm tra tính giờ để cải thiện kĩ năng quản lý thời gian cũng như xem lại các phần kiến thức nâng cao để cải thiện khả năng tư duy trả lời các câu hỏi khó. Bạn đã làm rất tốt, nhưng hoàn toàn có thể hoàn hảo hơn nữa!"
    elif predicted == 9:
        return "Bạn đang làm khá tốt ở các câu hỏi có độ khó Medium, tuy nhiên lại sai ở các câu hỏi có độ khó được đánh giá ở mức Easy và Hard. Nguyên nhân có thể là do sự thiếu cẩn thận trong quá trình làm bài. Để cải thiện, hãy thử làm bài một cách chậm rãi để cải thiện độ chính xác cho các câu hỏi Easy, nắm thật chắc bản chất của vấn đề và học kĩ lý thuyết, sau đó hãy luyện tập các câu hỏi có độ khó Hard để nâng cao khả năng giải quyết vấn đề của mình."
    elif predicted == 10:
        return "Bạn đã nắm khá chắc các kiến thức cơ bản và có thể xử lý các câu hỏi có độ khó được đánh giá ở mức Easy và Medium rất tốt, tuy nhiên mức độ thể hiện ở các câu có độ khó Hard đang chỉ ở mức trung bình khá. Kết quả này đã rất tốt, tuy nhiên bạn hoàn toàn có thể làm tốt hơn nữa! Hãy luyện tập thêm các câu hỏi có mức độ khó được đánh giá Hard để cải thiện tư duy và kĩ năng giải quyết các bài toán khó, đặc biệt tập trung vào việc hiểu bản chất của vấn đề và bạn hoàn toàn có thể đạt điểm tối đa."
    else:
        return "Có vẻ bạn chưa có quá nhiều tiếp xúc với toán SAT. Nhưng đừng lo, hãy bắt tay vào việc học luôn nào! Hãy nhớ rằng toán SAT không hề khó, bạn chỉ cần hiểu bản chất của vấn đề và liên hệ nó với các dạng bài bạn gặp, các công thức bạn đã học và bạn hoàn toàn có thể đạt được số điểm mong muốn! Hãy cố gắng luyện tập và học hỏi nhiều hơn. "

In [26]:
# New input data
new_input = np.array([[1, 1, 1]])

# Apply the same scaling using the fitted scaler
new_input_scaled = scaler_diff.transform(new_input)

# Predict the class using the trained model
predicted_diff = lg_diff_model.predict(new_input_scaled)

feedback_diff(predicted_diff[0])

'Về mặt tổng quát, bạn đã làm rất tốt ở các câu hỏi thuộc cả 3 độ khó Easy, Medium và Hard. Để có thể đạt được kết quả tốt thường xuyên hơn, bạn có thể luyện tập với các bài kiểm tra tính giờ để cải thiện kĩ năng quản lý thời gian cũng như xem lại các phần kiến thức nâng cao để cải thiện khả năng tư duy trả lời các câu hỏi khó. Bạn đã làm rất tốt, nhưng hoàn toàn có thể hoàn hảo hơn nữa!'

In [27]:
def feedback_dom(predicted):
    print("Cụ thể hơn nữa về các domain toán: ")
    if predicted == 0:
        return " Trong các phần kiến thức, bạn đang gặp khó khăn hầu hết ở phần Algebra và Geometry. Để cải thiện, hãy cố gắng ghi nhớ các công thức, hiểu bản chất dạng bài và tránh sai sót trong tính toán nhé! "
    elif predicted == 1:
        return " Các phần kiến thức Algebra và Advanced Math đã được bạn giải quyết rất tốt, tuy nhiên bạn lại đang yếu ở phần Problem-solving & Data Analysis và mắc một vài lỗi trong phần Geometry. Hãy xem lại video bài giảng để hiểu rõ hơn các dạng bài này nhé! "
    elif predicted == 2:
        return " Trong các phần kiến thức, bạn đã nắm chắc được kĩ năng của phần Problem-solving & Data Analysis, tuy nhiên vẫn còn mắc một vài lỗi trong phần Geometry và Advanced Math, và còn khá yếu phần Algebra. Để cải thiện, hãy xem lại video bài giảng để hiểu rõ hơn các dạng bài này nhé! "
    elif predicted == 3:
        return " Phần Algebra của bạn còn cần phải cải thiện, nhưng những domain còn lại là Advanced Math, Problem-solving & Data Analysis và Geometry đã được bạn giải quyết rất tốt. Hãy giữ vững phong độ này và cố gắng trau dồi kiến thức Algebra nhé! "
    elif predicted == 4:
        return " Kĩ năng Algebra và Geometry của bạn đang mới ở mức khá, trong khi Advanced Math và Problem-solving & Data Analysis còn khá yếu. Hãy xem những bài giảng để hiểu kĩ hơn về bản chất cũng như cách giải quyết các bài toán SAT nhé! "
    elif predicted == 5:
        return " Các phần kiến thức Algebra, Advanced math và Problem-Solving & Data Analysis của bạn đang ở mức khá tốt, tuy nhiên cần cải thiện thêm. Trong khi đó bạn đã làm rất tốt phần Geometry. Hãy tiếp tục phát huy và bổ sung các phần kiến thức mà bạn chưa nắm chắc nhé!"
    elif predicted == 6:
        return " Bạn đã làm rất tốt phần Algebra và Problem-solving & Data Analysis, tuy nhiên còn yếu phần Geometry và Advanced Math. Hãy xem lại video bài giảng để hiểu rõ hơn các dạng bài này nhé! "
    elif predicted == 7:
        return " 2 phần Algebra và Math bạn đã làm rất tốt, tuy nhiên bạn còn mắc một vài sai sót trong phần Problem-solving & Data Analysis và Geometry. Để hiểu kĩ hơn các dạng bài này, hãy xem lại video bài giảng nhé! "
    elif predicted == 8:
        return " Bạn làm rất tốt ở phần Algebra, tuy nhiên lại yếu các phần còn lại ( Advanced Math, Problem-solving & Data Analysis và Geometry ). Hãy tiếp tục phát huy thế mạnh của mình và bổ sung thêm kiến thức các phần bạn còn yếu nhé! "
    elif predicted == 9:
        return " Bạn còn yếu ở phần Algebra và Problem-solving & Data Analysis, ngoài ra phần Geometry bạn vẫn mới chỉ ở mức khá. Tuy nhiên bạn đã khá thành thạo phần Advanced Math. Để cải thiện, hãy cố gắng ghi nhớ các công thức, hiểu bản chất các dạng bài và tránh sai sót trong tính toán nhé! "
    elif predicted == 10:
        return " Bạn làm rất tốt ở phần Geometry, tuy nhiên lại yếu các phần còn lại ( Advanced Math, Problem-solving & Data Analysis và Algebra ). Hãy tiếp tục phát huy thế mạnh của mình và bổ sung thêm kiến thức các phần bạn còn yếu nhé! "
    elif predicted == 11:
        return " Các bài toán Sat đều có dạng của riêng chúng. Các dạng được chia làm 4 domain Algebra, Advanced Math, Problem-solving & Data Analysis. Bạn cần phải hiểu rõ từng dạng bài, cách giải và cách áp dụng công thức. Tuy bây giờ bạn còn yếu, nhưng điều đó hoàn toàn có thể thay đổi nên hãy cố gắng lên! "
    elif predicted == 12:
        return " Bạn đã làm rất tốt phần Algebra và Advanced Math, tuy nhiên còn yếu phần Geometry và Problem-solving & Data Analysis. Hãy xem lại video bài giảng để hiểu rõ hơn các dạng bài này nhé! "
    elif predicted == 13:
        return " Bạn làm rất tốt ở phần Geometry, tuy nhiên lại yếu các phần còn lại ( Advanced Math, Problem-solving & Data Analysis và Algebra ). Hãy tiếp tục phát huy thế mạnh của mình và bổ sung thêm kiến thức các phần bạn còn yếu nhé! "
    elif predicted == 14:
        return "Bạn làm rất tốt ở phần Advanced Math, tuy nhiên lại yếu các phần còn lại ( Problem-solving & Data Analysis, Geometry và Algebra ). Hãy tiếp tục phát huy thế mạnh của mình và bổ sung thêm kiến thức các phần bạn còn yếu nhé! "
    elif predicted == 15:
        return " Bạn đã làm rất tốt phần Advanced Math và Problem-solving & Data Analysis, tuy nhiên còn yếu phần Geometry và Algebra. Hãy xem lại video bài giảng để hiểu rõ hơn các dạng bài này nhé! "
    elif predicted == 16:
        return " Bạn đã làm rất tốt 3/4 phần kiến thức ( Algebra, Advanced Math và Geometry ). Tuy nhiên bạn còn đang yếu ở phần Problem-solving & Data Analysis. Hãy xem lại video bài giảng, tìm hiểu về logic của đề cũng như ghi nhớ các công thức để có thể đạt được điểm cao nhé! "
    elif predicted == 17:
        return " Bạn làm rất tốt ở phần Problem-solving & Data Analysis, tuy nhiên lại yếu các phần còn lại ( Advanced Math, Geometry và Algebra ). Hãy tiếp tục phát huy thế mạnh của mình và bổ sung thêm kiến thức các phần bạn còn yếu nhé! "
    elif predicted == 18:
        return " Bạn đã làm rất tốt ở cả 4 phần kiến thức ( Algebra, Advanced Math, Problem-solving & Data Analysis và Geometry ). Tuy nhiên đừng chủ quan nhé! Hãy cố gắng trau dồi nhiều kiến thức nhất có thể và tránh sai những lỗi cơ bản, và bạn hoàn toàn có thể đạt được mức điểm mong muôn! "
    elif predicted == 19:
        return " Bạn đã làm khá tốt ở 2 phần Advanced Math và Geometry, tuy nhiên bạn vẫn còn sai một vài lỗi ở phần Problem-solving & Data Analysis và còn yếu phần Algebra. Hãy xem lại video bài giảng để hiểu rõ hơn các dạng bài này nhé! "
    elif predicted == 20:
        return " Bạn đã làm rất tốt 3/4 phần kiến thức ( Algebra, Geometry và Problem-solving & Data Analysis ). Tuy nhiên bạn còn đang yếu ở phần Advanced Math. Hãy xem lại video bài giảng, ghi nhớ các công thức cũng như tìm hiểu thêm các dạng phương trình và đồ thị, cố gắng liên kết các kiến thức được học với công thức để có thể đạt được điểm cao nhé! "
    elif predicted == 21:
        return " Bạn đã làm rất tốt 3/4 phần kiến thức ( Algebra, Advanced Math và Problem-solving & Data Analysis ). Tuy nhiên bạn còn đang yếu ở phần Geometry. Hãy xem lại video bài giảng, ghi nhớ các công thức cũng như tìm hiểu thêm các dạng bài hình học, cố gắng liên kết các kiến thức được học với công thức để có thể đạt được điểm cao nhé! "
    elif predicted == 22:
        return " Bạn đã làm rất tốt phần Algebra và Geometry, tuy nhiên còn yếu phần Advanced Math và Problem-solving & Data Analysis. Hãy xem lại video bài giảng để hiểu rõ hơn các dạng bài này nhé! "
    else: 
        return "Bạn đã làm rất tốt phần Geometry và Problem-solving & Data Analysis, tuy nhiên còn yếu phần Algebra và Advanced Math. Hãy xem lại video bài giảng để hiểu rõ hơn các dạng bài này nhé!"

In [28]:
# New input data
new_input = np.array([[1, 1, 1, 1]])

# Apply the same scaling using the fitted scaler
new_input_scaled = scaler_dom.transform(new_input)

# Predict the class using the trained model
predicted_dom = lg_dom_model.predict(new_input_scaled)

feedback_dom(predicted_dom)

Cụ thể hơn nữa về các domain toán: 


' Bạn đã làm rất tốt ở cả 4 phần kiến thức ( Algebra, Advanced Math, Problem-solving & Data Analysis và Geometry ). Tuy nhiên đừng chủ quan nhé! Hãy cố gắng trau dồi nhiều kiến thức nhất có thể và tránh sai những lỗi cơ bản, và bạn hoàn toàn có thể đạt được mức điểm mong muôn! '